# API Nakala

In [ ]:
# ANR experts
# author: @sardinecan
# date: 2022-12
# description: this Julia notebook allows to interact with the Nakala's API
# licence: CC-0

# @todo : gestion des erreurs (réponses server) ?
# @todo : écrire un fichier de log pour récupérer les identifiants des ressources crées.

## Packages

In [2]:
using Pkg
path = @__DIR__
Pkg.activate(path)
Pkg.status()

using CSV
using DataFrames
using HTTP
using JSON
using Dates
using FileIO
using Images

  Activating project at `~/files/dh/nakalia`


Status `~/files/dh/nakalia/Project.toml`
  [336ed68f] CSV v0.10.14
  [a93c6f00] DataFrames v1.6.1
  [5789e2e9] FileIO v1.16.3
  [cd3eb016] HTTP v1.10.8
  [02fcd773] ImageTransformations v0.10.1
  [916415d5] Images v0.26.1
  [682c06a0] JSON v0.21.4
  [ade2ca70] Dates


## Identifiants

In [2]:
credentials = CSV.read(joinpath(path, "credentials.csv"), DataFrame, header=1) #liste des utilisateurs
user = "tnakala" #choix de l'utilisateur (api test = tnakala)
usrCredentials = filter(:user => n -> n == user, credentials) #récupération des identifiants
apiKey = usrCredentials[1, :apikey] #clé API

"01234567-89ab-cdef-0123-456789abcdef"

## API
Pour travailler avec l'API test, modifiez la valeur de la variable `apitest` par `true`

In [3]:
apitest = true

if apitest == true
  apiurl = "https://apitest.nakala.fr"
else 
  apiurl = "https://api.nakala.fr"
end

"https://apitest.nakala.fr"

## Librairie de fonctions

In [4]:
newARGS = apiKey #passe l'apikey dans la librairie
include("functions.jl")

localARGS = String[]


addFileToData (generic function with 1 method)

## Collections

### Création d'une collection

Une fois les identifiants chargés et l'API sélectionnée, entrez simplement un nom de collection pour la créer.

In [5]:
collectionName = "testCollection"
postCollection(collectionName)

Dict{String, Any} with 3 entries:
  "message" => "Collection created"
  "payload" => Dict{String, Any}("id"=>"10.34847/nkl.11afsbvn")
  "code"    => 201

### Ajouter des données à une collection

In [ ]:
collectionIdentifier = "10.34847/nkl.5c519k06"
datas = [
  "nakalaDataId_1",
  "nakalaDataId_2",
  "nakalaDataId_3"
]

postDatasToCollection(collectionIdentifier, datas)

### Supprimer des données d'une collection

In [ ]:
collectionIdentifier = ""
datas = [
  "nakalaDataId_1",
  "nakalaDataId_2",
  "nakalaDataId_3"
]

deleteDatasFromCollection(collectionIdentifier, datas)

## Utilisateurs

### Mes Informations

In [6]:
getUserInfo() #retourne un dictionnaire

# pour accéder aux entrées du dictionnaire
#username = getUserInfo()["username"]
#userGroupId = getUserInfo()["userGroupId"]

Dict{String, Any} with 11 entries:
  "firstLogin"  => "2020-03-18T11:20:00+01:00"
  "username"    => "tnakala"
  "surname"     => "Nakala"
  "userGroupId" => "26cef362-5bef-11eb-99d1-5254000a365d"
  "lastLogin"   => "2024-05-26T10:14:59+02:00"
  "mail"        => "nakala@huma-num.fr"
  "photo"       => "nakala@huma-num.fr"
  "fullname"    => "Test Nakala"
  "givenname"   => "Test"
  "apiKey"      => "01234567-89ab-cdef-0123-456789abcdef"
  "roles"       => Any["ROLE_USER"]

### Ressources utilisateur

In [8]:
scope = ["deposited", "owned", "shared", "editable", "readable", "all"]
#deposited : les données déposées par l'utilisateur (ROLE_DEPOSITOR)
#owned : les données dont l'utilisateur est propriétaire (ROLE_OWNER)
#shared : les données partagées avec l'utilisateur (ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER, mais pas ROLE_OWNER)
#editable : les données modifiables par l'utilisateur (ROLE_OWNER, ROLE_ADMIN ou ROLE_EDITOR)
#readable : les données lisibles par l'utilisateur (ROLE_OWNER, ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER)
#all : toute
postUserDatas(scope[1])

Dict{String, Any} with 2 entries:
  "totalRecords" => 108
  "data"         => Any[Dict{String, Any}("isDepositor"=>true, "isOwner"=>true,…

### Lister les données privées/publiques

In [ ]:
scope = ["deposited", "owned", "shared", "editable", "readable", "all"]
#deposited : les données déposées par l'utilisateur (ROLE_DEPOSITOR)
#owned : les données dont l'utilisateur est propriétaire (ROLE_OWNER)
#shared : les données partagées avec l'utilisateur (ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER, mais pas ROLE_OWNER)
#editable : les données modifiables par l'utilisateur (ROLE_OWNER, ROLE_ADMIN ou ROLE_EDITOR)
#readable : les données lisibles par l'utilisateur (ROLE_OWNER, ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER)
#all : toute
status = [ ["pending"], ["published"] ] #[ "pending", "published" ]

postUserDatas(scope[1], status[2])

### Changer le statut d'une donnée

In [ ]:
scope = ["deposited", "owned", "shared", "editable", "readable", "all"]
#deposited : les données déposées par l'utilisateur (ROLE_DEPOSITOR)
#owned : les données dont l'utilisateur est propriétaire (ROLE_OWNER)
#shared : les données partagées avec l'utilisateur (ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER, mais pas ROLE_OWNER)
#editable : les données modifiables par l'utilisateur (ROLE_OWNER, ROLE_ADMIN ou ROLE_EDITOR)
#readable : les données lisibles par l'utilisateur (ROLE_OWNER, ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER)
#all : toute
status = [ ["pending"], ["published"], ["moderated"] ]

datasToModify = get(postUserDatas(scope[1], status[1]), "data", "")
datasToModifyIds = [get(data, "identifier", "") for data in datasToModify]
newStatus = status[2][1] #pour récupérer la valeur textuelle du vecteur.

[putDataStatus(data, newStatus) for data in datasToModifyIds]

### Lister les données (titre - id)

In [11]:
scope = ["deposited", "owned", "shared", "editable", "readable", "all"]
#deposited : les données déposées par l'utilisateur (ROLE_DEPOSITOR)
#owned : les données dont l'utilisateur est propriétaire (ROLE_OWNER)
#shared : les données partagées avec l'utilisateur (ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER, mais pas ROLE_OWNER)
#editable : les données modifiables par l'utilisateur (ROLE_OWNER, ROLE_ADMIN ou ROLE_EDITOR)
#readable : les données lisibles par l'utilisateur (ROLE_OWNER, ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER)
#all : toute
status = [ ["pending"], ["published"] ]

datas = get(postUserDatas(scope[1]), "data", "nothing")

getDatasResume(datas)

13-element Vector{Any}:
 Dict("testdata" => "10.34847/nkl.5f638bd5")
 Dict("test titre" => "10.34847/nkl.ab5dp358")
 Dict("Comic émouvant n°1" => "10.34847/nkl.3cb08l7j")
 Dict("Test temporal" => "10.34847/nkl.06cb7lbf")
 Dict("test donnee 1258" => "10.34847/nkl.1db755kg")
 Dict("test 8" => "10.34847/nkl.baba7u44")
 Dict("27" => "10.34847/nkl.26aaf477")
 Dict("Nombre de subject" => "10.34847/nkl.80f1r8br")
 Dict("test za" => "10.34847/nkl.4ee5n2ct")
 Dict("test" => "10.34847/nkl.ec59s0vl")
 Dict("data loss" => "10.34847/nkl.3ddc101o")
 Dict("test pub" => "10.34847/nkl.7a3dn519")
 Dict("textes" => "10.34847/nkl.38dfb7by")

### Chercher une donnée précise

In [ ]:
title = "test"

scope = ["deposited", "owned", "shared", "editable", "readable", "all"]
#deposited : les données déposées par l'utilisateur (ROLE_DEPOSITOR)
#owned : les données dont l'utilisateur est propriétaire (ROLE_OWNER)
#shared : les données partagées avec l'utilisateur (ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER, mais pas ROLE_OWNER)
#editable : les données modifiables par l'utilisateur (ROLE_OWNER, ROLE_ADMIN ou ROLE_EDITOR)
#readable : les données lisibles par l'utilisateur (ROLE_OWNER, ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER)
#all : toute
status = [ ["pending"], ["published"] ]

datas = get(postUserDatas(scope[1], status[2], title), "data", "nothing")
datas[1] # il peut y avoir plusieurs résultats

### Télécharger les images d'une donnée

In [14]:
title = "Test temporal"

scope = ["deposited", "owned", "shared", "editable", "readable", "all"]
#deposited : les données déposées par l'utilisateur (ROLE_DEPOSITOR)
#owned : les données dont l'utilisateur est propriétaire (ROLE_OWNER)
#shared : les données partagées avec l'utilisateur (ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER, mais pas ROLE_OWNER)
#editable : les données modifiables par l'utilisateur (ROLE_OWNER, ROLE_ADMIN ou ROLE_EDITOR)
#readable : les données lisibles par l'utilisateur (ROLE_OWNER, ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER)
#all : toute
status = [ ["pending"], ["published"] ]

datas = get(postUserDatas(scope[1], status[2], title), "data", "nothing")
data = datas[1]

# peut fonctionner avec d'autres formats de données (mais pas toutes, voir https://juliaio.github.io/FileIO.jl/stable/registry/#Registry-table)

downloadFiles(data, title)

1-element Vector{Any}:
 "https://apitest.nakala.fr/data/" ⋯ 31 bytes ⋯ "b6c80b2c5c9710d28577106c5a16b6e"

### Chercher un fichier précis dans une donnée

In [16]:
title = "test"
filenames = ["The_Earth_seen_from_Apollo_17.jpg"]

scope = ["deposited", "owned", "shared", "editable", "readable", "all"]
#deposited : les données déposées par l'utilisateur (ROLE_DEPOSITOR)
#owned : les données dont l'utilisateur est propriétaire (ROLE_OWNER)
#shared : les données partagées avec l'utilisateur (ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER, mais pas ROLE_OWNER)
#editable : les données modifiables par l'utilisateur (ROLE_OWNER, ROLE_ADMIN ou ROLE_EDITOR)
#readable : les données lisibles par l'utilisateur (ROLE_OWNER, ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER)
#all : toute
status = [ ["pending"], ["published"] ]

datas = get(postUserDatas(scope[1], status[2], title), "data", "nothing")
data = datas[1]

getFilesUrlFromData(data, filenames)

1-element Vector{Any}:
 "https://apitest.nakala.fr/data/" ⋯ 31 bytes ⋯ "b6c80b2c5c9710d28577106c5a16b6e"

## Données

### Publier une donnée avec des fichiers

In [ ]:
path = joinpath(@__DIR__, "datasToSubmit") # chemin vers le dossier contenant les données à envoyer
directories = listFile(path)
[submitDataFromFolder(path, directory) for directory in directories]

### Ajouter des fichiers à une données

In [ ]:
dataIdentifier = "10.34847/nkl.5b62qk50" # identifier de la données à compléter
path = joinpath(@__DIR__, "filesToUpload") # chemin vers le dossier contenant les fichiers à ajouter
postedfiles = [postFile(joinpath(path, file)) for file in readdir(path)]

[addFileToData(dataIdentifier, file) for file in postedfiles]